Run GNPS data through LDA
=============================================

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from IPython.display import display, HTML
import pylab as plt

import os
import sys
basedir = '/Users/joewandy/git/lda/code'
sys.path.append(basedir)

from lda import VariationalLDA
from gnps import *

import warnings
warnings.filterwarnings('ignore')

<h2>1. GNPS Pipeline</h2>

Load the GNPS data into a `collection`. This is a list of Documents. Each Document has peaks in it (plus other metadata). Both the Document and Peak classes are defined inside gnps.py.

In [2]:
data_dir = '/Users/joewandy/Dropbox/Analysis/LDA/fingerid-104-traindata'

In [3]:
collection = load_gnps_data(os.path.join(data_dir, 'spectra/CCMSLIB*.ms'))

Total files = 5770
Loading 1000/5770
Loading 2000/5770
Loading 3000/5770
Loading 4000/5770
Loading 5000/5770


Here’s the agreed pipeline.
1. Threshold @ 500
2. Merge collision energy levels (7ppm)
3. Make priority queue of features [I only consider losses greater than 10 Da and less than 200Da]
4. Merge features using ‘Joe’ method with 7ppm gap
5. Keep all features (i.e. no longer remove those that only appear once)
6. Save
7. Normalise when we run VBLDA (level doesn’t really matter, but I’m doing 100.0)

### a. Threshold @ 500

In [4]:
min_intensity = 500

before = get_no_of_peaks(collection)
filter_spectra(collection, min_intensity)
after = get_no_of_peaks(collection)

print 'Total peaks before = %d' % before
print 'Total peaks after = %d' % after
print 'Total peaks filtered = %d' % (before-after)

Total peaks before = 15630923
Total peaks after = 588165
Total peaks filtered = 15042758


### b. Merge collision energy levels (7ppm)

In [5]:
merge_tol = 7
merge_spectra(collection, merge_tol)

Merging 1000/5770
Merging 2000/5770
Merging 3000/5770
Merging 4000/5770
Merging 5000/5770


### c. Make priority queue of features, d. Merge features using ‘Joe’ method with 7ppm gap

Here we still convert the collection into pandas dataframes in the old format, so we can run it through the old feature extraction codes. Need to improve this step.

In [15]:
params = {
    'fragment_grouping_tol' : 7,
    'loss_grouping_tol' : 7,
    'loss_threshold_min_val' : 10,
    'loss_threshold_max_val' : 200,
}
corpus, vocab, ms1, ms2 = get_corpus(collection, params)

Loading MS1 dataframe 5770 X 6
Loading MS2 dataframe 294050 X 6
Processing fragments for file 0
Total groups=43686
Processing losses for file 0
Total groups=20754
Populating the counts


In [20]:
save_object(corpus, os.path.join(data_dir, 'gnps.corpus'))

## 3. Run LDA

In [21]:
corpus = load_object(os.path.join(data_dir, 'gnps.corpus'))

In [23]:
K = 300
alpha = 1
eta = 0.1
level = 100.0
lda = VariationalLDA(corpus=corpus, K=K, alpha=alpha, eta=eta, update_alpha=True, normalise=level)

Found 64440 unique words
Object created with 3334 documents
Normalising intensities


In [24]:
lda.run_vb(n_its=1000, initialise=True)

Initialising
Starting iterations
Iteration 0 (change = 364.530585468) (16.436094 seconds, I think I'll finish in 273.9349 minutes)
Iteration 1 (change = 11.0493562323) (16.093172 seconds, I think I'll finish in 267.9513138 minutes)
Iteration 2 (change = 9.49760588424) (16.245285 seconds, I think I'll finish in 270.2132405 minutes)
Iteration 3 (change = 8.17880543952) (16.185492 seconds, I think I'll finish in 268.9489254 minutes)
Iteration 4 (change = 7.45180251364) (16.483307 seconds, I think I'll finish in 273.6228962 minutes)
Iteration 5 (change = 7.38991883319) (16.61388 seconds, I think I'll finish in 275.51351 minutes)
Iteration 6 (change = 8.09337029407) (16.307234 seconds, I think I'll finish in 270.156509933 minutes)
Iteration 7 (change = 10.1966222732) (16.233567 seconds, I think I'll finish in 268.66553385 minutes)
Iteration 8 (change = 16.1548991453) (16.215072 seconds, I think I'll finish in 268.0891904 minutes)
Iteration 9 (change = 25.4921496665) (16.289536 seconds, I th

In [25]:
save_object(lda, os.path.join(data_dir, 'gnps.lda'))

## 4. Visualisation

In [26]:
for k in range(lda.K):
    
    # td is a dictionary, the key is a word, the value is the probability
    td = lda.get_topic_as_dict(k) 
    
    # convert td into a list of (word, prob)
    items = td.items()
    
    # sort items by the probabilities
    # itemgetter(1) refers to the 2nd field of each element, used for sorting
    sorted_items = sorted(items, key=itemgetter(1), reverse=True)
    
    # print the top-10 words
    # alternatively we can also set a threshold t and only print words at probabilities > t
    print 'Topic %d' % k
    for word, prob in sorted_items[0:10]:
        print ' -', word, prob
    print

Topic 0
 - fragment_758.44849 0.00463062312585
 - fragment_828.48658 0.0046305192345
 - loss_71.08619 0.00463029788638
 - loss_169.03569 0.00424443648762
 - loss_131.02091 0.00422109817816
 - loss_182.08294 0.00420654353809
 - loss_195.04407 0.00405186888438
 - loss_86.01696 0.00385909066165
 - loss_194.03775 0.00385905848068
 - loss_123.03434 0.0038506518266

Topic 1
 - fragment_210.09712 0.0119177197404
 - fragment_100.11219 0.0115381328233
 - fragment_214.14418 0.0112759245262
 - fragment_186.14854 0.00970267202681
 - fragment_228.15909 0.0089160375914
 - fragment_197.12388 0.0083919825735
 - loss_113.08482 0.00770834423468
 - fragment_196.13286 0.00734318140203
 - fragment_168.13629 0.00681829628951
 - loss_27.99559 0.00681829628951

Topic 2
 - fragment_326.16897 0.00752886981835
 - loss_17.00784 0.00701641366249
 - fragment_327.15273 0.00701071220494
 - fragment_151.09511 0.0063141746292
 - loss_18.01087 0.00602671612696
 - fragment_421.18658 0.00493071962904
 - fragment_420.18897